In [1]:
import shap
import lime
import lime.lime_image
from skimage.segmentation import mark_boundaries
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
from tensorflow.keras.models import Model
import tensorflow as tf
import os
import cv2
import numpy as np
import tensorflow as tf
from keras import backend as K

c:\Users\wfaiz\anaconda3\envs\pcb_proj\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
model_path = os.path.join(parent_dir, 'models', "model_enhanced_unet_100.keras")

In [3]:
# Load the model
model = tf.keras.models.load_model(model_path)

In [4]:
# load test set data
X_test_path =  os.path.join(parent_dir, 'models', "X_test.npy")
X_test = np.load(X_test_path)

In [5]:
# load train set data
X_train_path =  os.path.join(parent_dir, 'models', "x_train_filtered_images.npy")
X_train = np.load(X_train_path)

In [6]:
# Define the image path
path = 'C:/users/wfaiz/Data_Science/PCB_Defect/data_full/Images/'


In [7]:
# Loading and cropping image as per model input dimensions
def loading_image(path, image, crop_size):
    image_path = os.path.join(path, image)
    image_test = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  
    image_test = cv2.resize(image_test, (600, 600))  

    # Split the image into 100x100
    
    crops = []
    for i in range(0, 600, crop_size):
        for j in range(0, 600, crop_size):
            crop = image_test[i:i+crop_size, j:j+crop_size]
            crop = np.asarray(crop, dtype=np.float32) / 255.0  # Normalize
            crop = np.expand_dims(crop, axis=-1)  # Add channel dimension
            crops.append(crop)
    return image_test, crops

In [8]:
img = "l_light_11_spur_02_5_600.jpg"

In [9]:
crop_size = 100
image, crops = loading_image(path, img, crop_size)

In [11]:
# Background data for SHAP
background = X_train[np.random.choice(X_train.shape[0], 10, replace=False)]

# Initialize SHAP DeepExplainer
explainer = shap.DeepExplainer(model, background)

# Calculate SHAP values for a subset of test data
shap_values = explainer.shap_values(X_test[:5])

# Visualize the SHAP values
shap.summary_plot(shap_values, X_test[:5])

InvalidArgumentError: {{function_node __wrapped__Pack_N_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [10,100,100,1] != values[1].shape = [10,7] [Op:Pack] name: packed

In [12]:
shap_values = explainer_shap.shap_values(X_test)

NameError: name 'explainer_shap' is not defined

In [13]:
print("Output type:", type(model.output[1]))
print("Output shape:", model.output[1].shape)

Output type: <class 'keras.src.backend.common.keras_tensor.KerasTensor'>
Output shape: (None, 7)


In [14]:
def lime_predict(images):
    
    if images.shape[-1] == 3:
        images = rgb2gray(images)  # Convert RGB to grayscale
        
    if images.ndim == 3:
        images = images[..., np.newaxis]  # Add channel dimension

    return model.predict(images)

In [16]:
explainer = lime.lime_image.LimeImageExplainer()

In [18]:
explanation = explainer.explain_instance(crops[0], 
                                         lime_predict, 
                                         top_labels=5, 
                                         hide_color=0, 
                                         num_samples=100)

# Display the explanation
temp, mask = explanation.get_image_and_mask(explanation.top_labels[0], 
                                            positive_only=False, 
                                            num_features=10, 
                                            hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))
plt.show()

ValueError: Only RGB images can be converted to Lab space.